# 목적

다시 돌아온 VGG.

간단한 모델도 어느정도 정확도가 보장될 줄 알았는데, 최대값이 30%라는게 좀 충격적이다.

첫번째로 순수하게 VGG19를 바닥부터 구현해서 돌려본다.
시간이 정말 많이 들 것 같은 예감이 든다.

# 0.1. 구글 드라이브 연동

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 0.2. 라이브러리 임포트 및 전역변수 설정

In [2]:
from pathlib import Path
from easydict import EasyDict as edict
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
from collections import Counter
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
root = '/gdrive/My Drive/ctp431'
inst_pool = ['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']

# Step 1. VGG19 모델 만들기

In [4]:
class VGG19_Pure(nn.Module):
    def __init__(self, num_class = 11):
        super(VGG19_Pure, self).__init__()

        self.conv_block = nn.Sequential(
            # block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 5
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.fc_block = nn.Sequential(
            # block 6
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

        self.fc_block_test = nn.Sequential(
            # block 6
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block_test(x)
        return x

# Step 2. 학습 돌리기

In [5]:
# 데이터 로드
test_data_path = root + "/MK2/DataSet/test_data.pt"
test_x, test_y = torch.load(test_data_path)

train_data_path = root + "/MK2/DataSet/training_data.pt"
train_x, train_y = torch.load(train_data_path)

# 데이터셋과 DataLoader 생성
train_data = TensorDataset(train_x, train_y)
test_data = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_data, batch_size=35, shuffle=True)
val_loader = DataLoader(test_data, batch_size=35, shuffle=False)

# 클래스 분포 확인
print(train_y.unique(), train_y.bincount())
print(test_y.unique(), test_y.bincount())

<ipython-input-5-8f30f2df9be4>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_x, test_y = torch.load(test_data_path)
<ipython-input-5-8f30f2df9be4>:6: FutureWarning: 

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]) tensor([280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]) tensor([70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70])


In [8]:
# 모델, 손실 함수, 옵티마이저 초기화
model = VGG19_Pure(num_class=11)
#optimizer = optim.Adam(model.parameters(), lr=0.001) <- sucks!
#optimizer =optim.Adam(model.parameters(), lr=1e-4) <- 40%
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=5e-4)

def initialize_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(initialize_weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

result_dir = Path(root) / 'MK2/Results/VGG19_Pure'
result_dir.mkdir(parents=True, exist_ok=True)

# Create directory name.
num_trial=0
parent_dir = result_dir / f'trial_{num_trial}'
while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
parent_dir.mkdir(parents=True, exist_ok=True)
print(f'Trial save path : {parent_dir}')

# 트레이닝 루프
epochs = 15
for epoch in tqdm(range(epochs)):
    print("\n")
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = nn.CrossEntropyLoss()(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            print(predicted)
            print(labels)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Accuracy: {val_accuracy:.2f}%")


model_save_path = str(parent_dir) + '/vgg19_pure_model.pt'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Trial save path : /gdrive/My Drive/ctp431/MK2/Results/VGG19_Pure/trial_13


  0%|          | 0/15 [00:00<?, ?it/s]



Epoch 1/15, Loss: 2.5497
tensor([ 3,  1,  0,  0,  0,  0,  6,  1,  0,  0,  0,  1,  0,  1,  1,  3,  1,  0,
         0,  0,  1,  1,  1,  0, 10,  0,  0,  4,  6,  4,  4,  8,  4,  8,  4],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([8, 0, 4, 1, 1, 1, 4, 1, 0, 1, 1, 4, 1, 0, 0, 1, 1, 1, 4, 1, 1, 1, 8, 3,
        1, 0, 0, 0, 1, 0, 8, 0, 4, 4, 5], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 8, 1, 1, 1, 1, 1, 1, 8, 1, 1, 8, 1, 8, 1, 1, 8, 1, 1, 1, 1, 1,
        1, 1, 8, 0, 8, 1, 1, 0, 0, 0, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([ 8,  0,  1,  1,  0,  1,  8,  4,  1,  4,  4,  0,  1,  8,  8,  8,  1,  4,
         1,  4,  

  7%|▋         | 1/15 [01:04<15:07, 64.81s/it]

tensor([ 0,  5,  4, 10,  4,  3, 10,  4,  4,  4,  0,  8,  4,  0, 10,  4,  0,  4,
         4,  4,  4,  4,  4,  0,  6,  0,  4,  5,  4,  1,  4,  4,  4,  4,  5],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
Validation Loss: 2.3090, Accuracy: 17.14%


Epoch 2/15, Loss: 2.2082
tensor([ 0,  1, 10,  1,  0, 10,  1,  1,  0,  2,  1,  1,  2,  1,  2,  1,  2,  2,
         1,  0,  1,  1,  1,  0, 10,  4, 10, 10,  1, 10, 10,  2, 10,  1, 10],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([ 2, 10, 10,  6,  1,  1, 10,  1,  0,  1,  6, 10,  1,  2,  0,  1,  1,  1,
         2,  1,  1,  1, 10,  7,  2,  0, 10,  2,  1,  1,  2,  2, 10,  4, 10],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

...

 93%|█████████▎| 14/15 [15:01<01:04, 64.40s/it]

tensor([ 3,  9, 10, 10, 10,  3, 10, 10, 10,  4,  3,  4, 10,  3, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10,  3,  3, 10,  3, 10,  3,  4,  0,  5,  0, 10],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
Validation Loss: 2.0581, Accuracy: 38.57%


Epoch 15/15, Loss: 1.3258
tensor([ 0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  9,  3,  0,  3,  0,  1,
         0,  0,  0,  0,  3,  0, 10,  0,  0, 10,  3, 10,  0,  0,  0,  0,  0],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 9, 6, 1, 0, 0, 0, 0, 6, 1, 0, 1, 0, 0, 0, 0, 0, 8, 0, 0, 0, 1, 9,
        3, 0, 0, 0, 3, 3, 0, 0, 1, 4, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 15/15 [16:06<00:00, 64.41s/it]

tensor([ 3, 10, 10, 10, 10,  3, 10, 10,  3,  4,  3,  4, 10,  3, 10, 10,  3,  3,
        10, 10, 10, 10, 10, 10,  3,  3, 10, 10, 10,  3,  4,  4, 10,  0, 10],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
Validation Loss: 2.0278, Accuracy: 40.26%


Model saved to /gdrive/My Drive/ctp431/MK2/Results/VGG19_Pure/trial_13/vgg19_pure_model.pt
